In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Large_B-cell_Lymphoma"
cohort = "GSE248835"

# Input paths
in_trait_dir = "../../input/GEO/Large_B-cell_Lymphoma"
in_cohort_dir = "../../input/GEO/Large_B-cell_Lymphoma/GSE248835"

# Output paths
out_data_file = "../../output/preprocess/Large_B-cell_Lymphoma/GSE248835.csv"
out_gene_data_file = "../../output/preprocess/Large_B-cell_Lymphoma/gene_data/GSE248835.csv"
out_clinical_data_file = "../../output/preprocess/Large_B-cell_Lymphoma/clinical_data/GSE248835.csv"
json_path = "../../output/preprocess/Large_B-cell_Lymphoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Impact of Tumor Microenvironment on Efficacy of CD19 CAR T-Cell Therapy or Chemotherapy and Transplant in Large B-Cell Lymphoma"
!Series_summary	"The phase 3 ZUMA-7 trial in second-line large B-cell lymphoma demonstrated superiority of anti-CD19 CAR T-cell therapy (axicabtagene ciloleucel; axi-cel) over standard of care (SOC; salvage chemotherapy followed by hematopoietic transplantation). Here, we present a prespecified exploratory analysis examining the association between pretreatment tumor characteristics and the efficacy of axi-cel versus SOC. B-cell gene expression signature (GES) and CD19 expression significantly associated with improved event-free survival (EFS) for axi-cel (P=.0002 for B-cell GES; P=.0165 for CD19 expression) but not SOC (P=.9374 for B-cell GES; P=.5526 for CD19 expression). Axi-cel showed superior EFS over SOC irrespective of B-cell GES and CD19 expression (P=8.56e–9 for B-cell GES high; P=.0019 for B-cell GES low; P=3.8

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import os
import pandas as pd
import numpy as np
import json
from glob import glob

# 1. Gene Expression Data Availability
# This dataset includes gene expression data from the phase 3 ZUMA-7 trial in large B-cell lymphoma
# The background information suggests RNA sequencing for gene expression profiling
is_gene_available = True  # Gene expression data is available

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Looking at the Sample Characteristics Dictionary:

# For trait (Large B-cell Lymphoma):
# Row 1 shows treatment arm information which can be used to classify patients
trait_row = 1  # 'treatment arm' contains treatment information

# Age information is not available in the sample characteristics dictionary
age_row = None  # Age data is not available

# Gender information is not available in the sample characteristics dictionary
gender_row = None  # Gender data is not available

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """
    Convert treatment arm information to binary:
    Axicabtagene Ciloleucel = 1 (experimental treatment)
    Standard of Care Chemotherapy = 0 (control group)
    """
    if value is None or pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "Axicabtagene Ciloleucel" in value:
        return 1
    elif "Standard of Care Chemotherapy" in value:
        return 0
    else:
        return None

def convert_age(value):
    """
    Since age data is not available, this function is defined but won't be used.
    """
    return None

def convert_gender(value):
    """
    Since gender data is not available, this function is defined but won't be used.
    """
    return None

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Only proceed if trait_row is not None
if trait_row is not None:
    # List files in the cohort directory to help diagnose
    print(f"Files in the cohort directory ({in_cohort_dir}):")
    if os.path.exists(in_cohort_dir):
        files = os.listdir(in_cohort_dir)
        for file in files:
            print(f"  - {file}")
    else:
        print(f"  Directory not found")
    
    # Try to find clinical data from possible locations
    # First check if a variable containing clinical data is already available
    # Try accessing the clinical_data that might have been generated in a previous step
    try:
        # Assuming clinical_data might have been created in a previous step and is in memory
        if 'clinical_data' in locals() or 'clinical_data' in globals():
            print("Found clinical_data variable in memory")
            # Extract clinical features
            selected_clinical_df = geo_select_clinical_features(
                clinical_df=clinical_data,
                trait=trait,
                trait_row=trait_row,
                convert_trait=convert_trait,
                age_row=age_row,
                convert_age=convert_age,
                gender_row=gender_row,
                convert_gender=convert_gender
            )
            
            # Preview the data
            preview = preview_df(selected_clinical_df)
            print("Clinical Data Preview:")
            print(preview)
            
            # Create directory if it doesn't exist
            os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
            
            # Save the clinical data
            selected_clinical_df.to_csv(out_clinical_data_file, index=False)
            print(f"Clinical data saved to {out_clinical_data_file}")
        else:
            print("Clinical data variable not found in memory")
            # Let's search for other possible file names or locations
            clinical_files = glob(os.path.join(in_cohort_dir, "*clinical*.csv")) + \
                             glob(os.path.join(in_cohort_dir, "*sample*.csv")) + \
                             glob(os.path.join(in_cohort_dir, "*char*.csv"))
            
            if clinical_files:
                print(f"Found potential clinical data files: {clinical_files}")
                clinical_data = pd.read_csv(clinical_files[0])
                
                # Extract clinical features
                selected_clinical_df = geo_select_clinical_features(
                    clinical_df=clinical_data,
                    trait=trait,
                    trait_row=trait_row,
                    convert_trait=convert_trait,
                    age_row=age_row,
                    convert_age=convert_age,
                    gender_row=gender_row,
                    convert_gender=convert_gender
                )
                
                # Preview the data
                preview = preview_df(selected_clinical_df)
                print("Clinical Data Preview:")
                print(preview)
                
                # Create directory if it doesn't exist
                os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
                
                # Save the clinical data
                selected_clinical_df.to_csv(out_clinical_data_file, index=False)
                print(f"Clinical data saved to {out_clinical_data_file}")
            else:
                print("No clinical data files found in the cohort directory")
                
                # If no clinical file is found, let's try checking in in_trait_dir instead
                print(f"Checking for clinical data in trait directory: {in_trait_dir}")
                clinical_files = glob(os.path.join(in_trait_dir, "*clinical*.csv")) + \
                                 glob(os.path.join(in_trait_dir, "*sample*.csv")) + \
                                 glob(os.path.join(in_trait_dir, "*char*.csv"))
                
                if clinical_files:
                    print(f"Found potential clinical data files in trait directory: {clinical_files}")
                else:
                    print("No clinical data files found in the trait directory either")
                    print("Unable to locate clinical data. Skipping clinical feature extraction.")
    except Exception as e:
        print(f"Error during clinical data processing: {e}")
        print("Skipping clinical feature extraction due to error")


Files in the cohort directory (../../input/GEO/Large_B-cell_Lymphoma/GSE248835):
  - GSE248835_family.soft.gz
  - GSE248835_series_matrix.txt.gz
Found clinical_data variable in memory
Clinical Data Preview:
{'GSM7920866': [1.0], 'GSM7920867': [1.0], 'GSM7920868': [1.0], 'GSM7920869': [0.0], 'GSM7920870': [0.0], 'GSM7920871': [0.0], 'GSM7920872': [1.0], 'GSM7920873': [1.0], 'GSM7920874': [1.0], 'GSM7920875': [1.0], 'GSM7920876': [1.0], 'GSM7920877': [0.0], 'GSM7920878': [1.0], 'GSM7920879': [0.0], 'GSM7920880': [1.0], 'GSM7920881': [1.0], 'GSM7920882': [1.0], 'GSM7920883': [1.0], 'GSM7920884': [1.0], 'GSM7920885': [0.0], 'GSM7920886': [0.0], 'GSM7920887': [0.0], 'GSM7920888': [0.0], 'GSM7920889': [0.0], 'GSM7920890': [1.0], 'GSM7920891': [1.0], 'GSM7920892': [0.0], 'GSM7920893': [1.0], 'GSM7920894': [1.0], 'GSM7920895': [1.0], 'GSM7920896': [0.0], 'GSM7920897': [0.0], 'GSM7920898': [0.0], 'GSM7920899': [0.0], 'GSM7920900': [0.0], 'GSM7920901': [1.0], 'GSM7920902': [0.0], 'GSM7920903': [

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Add diagnostic code to check file content and structure
print("Examining matrix file structure...")
with gzip.open(matrix_file, 'rt') as file:
    table_marker_found = False
    lines_read = 0
    for i, line in enumerate(file):
        lines_read += 1
        if '!series_matrix_table_begin' in line:
            table_marker_found = True
            print(f"Found table marker at line {i}")
            # Read a few lines after the marker to check data structure
            next_lines = [next(file, "").strip() for _ in range(5)]
            print("First few lines after marker:")
            for next_line in next_lines:
                print(next_line)
            break
        if i < 10:  # Print first few lines to see file structure
            print(f"Line {i}: {line.strip()}")
        if i > 100:  # Don't read the entire file
            break
    
    if not table_marker_found:
        print("Table marker '!series_matrix_table_begin' not found in first 100 lines")
    print(f"Total lines examined: {lines_read}")

# 2. Try extracting gene expression data from the matrix file again with better diagnostics
try:
    print("\nAttempting to extract gene data from matrix file...")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {str(e)}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")

# If data extraction failed, try an alternative approach using pandas directly
if not is_gene_available:
    print("\nTrying alternative approach to read gene expression data...")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            # Skip lines until we find the marker
            for line in file:
                if '!series_matrix_table_begin' in line:
                    break
            
            # Try to read the data directly with pandas
            gene_data = pd.read_csv(file, sep='\t', index_col=0)
            
            if not gene_data.empty:
                print(f"Successfully extracted gene data with alternative method: {gene_data.shape}")
                print("First 20 gene IDs:")
                print(gene_data.index[:20])
                is_gene_available = True
            else:
                print("Alternative extraction method also produced empty data")
    except Exception as e:
        print(f"Alternative extraction failed: {str(e)}")


Examining matrix file structure...
Line 0: !Series_title	"Impact of Tumor Microenvironment on Efficacy of CD19 CAR T-Cell Therapy or Chemotherapy and Transplant in Large B-Cell Lymphoma"
Line 1: !Series_geo_accession	"GSE248835"
Line 2: !Series_status	"Public on Feb 26 2024"
Line 3: !Series_submission_date	"Nov 28 2023"
Line 4: !Series_last_update_date	"Jul 25 2024"
Line 5: !Series_pubmed_id	"38233586"
Line 6: !Series_summary	"The phase 3 ZUMA-7 trial in second-line large B-cell lymphoma demonstrated superiority of anti-CD19 CAR T-cell therapy (axicabtagene ciloleucel; axi-cel) over standard of care (SOC; salvage chemotherapy followed by hematopoietic transplantation). Here, we present a prespecified exploratory analysis examining the association between pretreatment tumor characteristics and the efficacy of axi-cel versus SOC. B-cell gene expression signature (GES) and CD19 expression significantly associated with improved event-free survival (EFS) for axi-cel (P=.0002 for B-cell GES;

### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers in the gene expression data
# Based on the output, the gene identifiers appear to be numbers (1, 2, 3, 4, 5, etc.)
# These are not standard human gene symbols, which would typically be names like BRCA1, TP53, CD19, etc.
# These numeric identifiers likely represent probe IDs from a microarray platform
# that need to be mapped to standard gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 210225 rows

Gene annotation preview (first few rows):
{'ID': ['1', '2', '3', '4', '5'], 'Gene_Signature_Name': ['TIS.IO360', 'APM.IO360', 'APM Loss.IO360', 'Apoptosis.IO360', 'ARG1.IO360'], 'ORF': [nan, nan, nan, nan, nan], 'GB_ACC': [nan, nan, nan, nan, nan], 'NS Probe ID': [nan, nan, nan, nan, nan], 'Analyte Type': ['IO360 Signature', 'IO360 Signature', 'IO360 Signature', 'IO360 Signature', 'IO360 Signature'], 'SPOT_ID': ['TIS.IO360', 'APM.IO360', 'APM Loss.IO360', 'Apoptosis.IO360', 'ARG1.IO360']}

Column names in gene annotation data:
['ID', 'Gene_Signature_Name', 'ORF', 'GB_ACC', 'NS Probe ID', 'Analyte Type', 'SPOT_ID']

The dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.
Number of rows with GenBank accessions: 768 out of 210225

The dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.
Example SPOT_ID format: TIS.IO360


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Analyze gene annotation data and determine mapping columns
print("\nAnalyzing gene annotation data to determine mapping columns...")

# Based on our observation, 'ID' column in gene_annotation matches the numeric indices in gene_data
# For gene symbols, we need to check which column contains valid gene symbols
# Looking at the preview, 'Gene_Signature_Name' appears to contain gene signature names

# Let's check both 'Gene_Signature_Name' and 'SPOT_ID' to see which one has proper gene symbols
# Display some unique values from both columns
print("Sample Gene_Signature_Name values:")
print(gene_annotation['Gene_Signature_Name'].dropna().unique()[:10])

print("\nSample SPOT_ID values:")
print(gene_annotation['SPOT_ID'].dropna().unique()[:10])

# Check if we have actual human gene symbols by looking at the data patterns
gene_symbols_count = gene_annotation['Gene_Signature_Name'].apply(lambda x: 
                                 bool(extract_human_gene_symbols(str(x)) if pd.notna(x) else False)).sum()
print(f"\nNumber of rows with potential human gene symbols in Gene_Signature_Name: {gene_symbols_count}")

spot_id_symbols_count = gene_annotation['SPOT_ID'].apply(lambda x: 
                                   bool(extract_human_gene_symbols(str(x)) if pd.notna(x) else False)).sum()
print(f"Number of rows with potential human gene symbols in SPOT_ID: {spot_id_symbols_count}")

# Based on the analysis, determine which columns to use for mapping
# We need to determine the appropriate column for gene symbols
# For this dataset, we'll use Gene_Signature_Name as it may contain the signature or gene names

# 2. Create the gene mapping dataframe
print("\nCreating gene mapping dataframe...")
mapping_df = get_gene_mapping(gene_annotation, 'ID', 'Gene_Signature_Name')
print(f"Created mapping dataframe with {len(mapping_df)} rows")
print("Mapping preview:")
print(preview_df(mapping_df))

# 3. Apply gene mapping to convert probe-level data to gene expression data
print("\nConverting probe-level measurements to gene expression data...")
# Fix: Use the gene_data variable we extracted earlier (in step 3) 
# rather than the undefined genetic_data
gene_expression_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Successfully created gene expression data with {len(gene_expression_data)} rows (genes)")

# Preview the gene expression data
print("\nGene expression data preview (first few genes):")
gene_preview = gene_expression_data.head(5)
print(f"Shape: {gene_preview.shape}")
print(preview_df(gene_preview))

# Check for actual genes in the resulting gene expression data
sample_genes = gene_expression_data.index.tolist()[:20]
print("\nSample genes in the resulting dataset:")
print(sample_genes)

# Save gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_expression_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")



Analyzing gene annotation data to determine mapping columns...
Sample Gene_Signature_Name values:
['TIS.IO360' 'APM.IO360' 'APM Loss.IO360' 'Apoptosis.IO360' 'ARG1.IO360'
 'B Cells.IO360' 'B7-H3.IO360' 'CD45.IO360' 'CD8 T Cells.IO360'
 'CTLA4.IO360']

Sample SPOT_ID values:
['TIS.IO360' 'APM.IO360' 'APM Loss.IO360' 'Apoptosis.IO360' 'ARG1.IO360'
 'B Cells.IO360' 'B7-H3.IO360' 'CD45.IO360' 'CD8 T Cells.IO360'
 'CTLA4.IO360']



Number of rows with potential human gene symbols in Gene_Signature_Name: 1072
Number of rows with potential human gene symbols in SPOT_ID: 48

Creating gene mapping dataframe...
Created mapping dataframe with 210225 rows
Mapping preview:
{'ID': ['1', '2', '3', '4', '5'], 'Gene': ['TIS.IO360', 'APM.IO360', 'APM Loss.IO360', 'Apoptosis.IO360', 'ARG1.IO360']}

Converting probe-level measurements to gene expression data...


Successfully created gene expression data with 789 rows (genes)

Gene expression data preview (first few genes):
Shape: (5, 256)
{'GSM7920866': [782.1839951, 3.83423527, 13.03639992, 8.435317594, 19.9380234], 'GSM7920867': [1621.002775, 5.25940108, 10.51880216, 69.75626698, 1246.754867], 'GSM7920868': [1909.47077, 3.5541568545, 21.32494113, 231.0201955, 9.773931349], 'GSM7920869': [1977.105386, 4.057492396, 35.41084273, 265.5813205, 18.44314725], 'GSM7920870': [992.555453, 1.771717389, 8.661729459, 266.1513234, 17.71717389], 'GSM7920871': [911.3641002, 9.550222605, 27.28635031, 603.0283417, 27.28635031], 'GSM7920872': [667.4939404, 3.024897011, 14.11618605, 112.9294884, 7.394192693], 'GSM7920873': [1006.014499, 5.051118825, 13.4696502, 142.2731803, 17.67891589], 'GSM7920874': [755.1144365, 5.53013236, 11.56300402, 445.9297638, 28.6561404], 'GSM7920875': [886.8469287, 2.3261558785, 5.815389696, 211.098646, 9.304623514], 'GSM7920876': [1070.110026, 2.624656686, 5.999215282, 217.471554, 1

### Step 7: Data Normalization and Linking

### Step 8: Data Normalization and Linking

In [8]:
# 1. Save the normalized gene expression data from the previous step
print("\nSaving normalized gene data...")
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Extract clinical data from the matrix file
print("\nExtracting clinical data...")
try:
    # Get the file paths again to make sure we have them
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    
    # Extract background information and clinical data
    background_info, clinical_data = get_background_and_clinical_data(
        matrix_file, 
        prefixes_a=['!Series_title', '!Series_summary', '!Series_overall_design'],
        prefixes_b=['!Sample_geo_accession', '!Sample_characteristics_ch1']
    )
    
    # Process clinical data using trait information from Step 2
    trait_row = 1  # Based on analysis in step 2 - group (OW/OB vs NW/MONW)
    gender_row = 0  # Gender data
    age_row = 2     # Age data
    
    # Define conversion functions based on Step 2
    def convert_trait(value):
        """Convert trait value (binary: 1 for OW/OB, 0 for NW/MONW)"""
        if pd.isna(value):
            return None
        
        # Extract value after colon if present
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        if 'OW/OB' in value:
            return 1  # Overweight/Obese is associated with higher LDL cholesterol
        elif 'NW' in value or 'MONW' in value:
            return 0  # Normal weight (includes metabolically obese normal weight)
        else:
            return None

    def convert_gender(value):
        """Convert gender value to binary (0: female, 1: male)"""
        if pd.isna(value):
            return None
        
        # Extract value after colon if present
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        # Convert gender
        if value.lower() == 'woman':
            return 0
        elif value.lower() == 'man':
            return 1
        else:
            return None
    
    def convert_age(value):
        """Convert age value to float"""
        if pd.isna(value):
            return None
        
        # Extract value after colon if present
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        try:
            return float(value)  # Convert to float for continuous variable
        except:
            return None
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    print(f"Clinical data shape: {selected_clinical_df.shape}")
    
    # Check if we have valid trait information
    is_trait_available = trait_row is not None and not selected_clinical_df.loc[trait].isnull().all()
    print(f"Trait information available: {is_trait_available}")
    
except Exception as e:
    print(f"Error extracting clinical data: {e}")
    is_trait_available = False
    selected_clinical_df = pd.DataFrame()

# 3. Link clinical and genetic data
print("\nLinking clinical and genetic data...")
try:
    if is_trait_available and not selected_clinical_df.empty:
        # Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
        print(f"Created linked data with {linked_data.shape[0]} samples and {linked_data.shape[1]} features")
    else:
        print("Cannot link data: clinical data is not available")
        linked_data = pd.DataFrame()
        is_trait_available = False
except Exception as e:
    print(f"Error linking clinical and genetic data: {e}")
    is_trait_available = False
    linked_data = pd.DataFrame()

# 4. Handle missing values in the linked data
if is_trait_available and not linked_data.empty:
    print("\nHandling missing values...")
    try:
        # Rename the first column to the trait name for consistency
        if linked_data.columns[0] != trait:
            linked_data = linked_data.rename(columns={linked_data.columns[0]: trait})
        
        linked_data = handle_missing_values(linked_data, trait)
        print(f"After handling missing values: {linked_data.shape[0]} samples and {linked_data.shape[1]} features")
    except Exception as e:
        print(f"Error handling missing values: {e}")
        
    # 5. Determine whether the trait and demographic features are biased
    print("\nEvaluating feature bias...")
    try:
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        print(f"Trait bias determination: {is_biased}")
        print(f"Final linked data shape: {linked_data.shape[0]} samples and {linked_data.shape[1]} features")
    except Exception as e:
        print(f"Error evaluating feature bias: {e}")
        is_biased = True
else:
    print("\nSkipping missing value handling and bias evaluation as linked data is not available")
    is_biased = True

# 6. Validate and save cohort information
print("\nPerforming final validation...")
note = ""
if not is_trait_available:
    note = "Dataset does not contain required trait information"
elif is_biased:
    note = "Dataset has severe bias in the trait distribution"

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 7. Save the linked data if usable
print(f"\nDataset usability for {trait} association studies: {is_usable}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Final linked data saved to {out_data_file}")
else:
    if note:
        print(f"Reason: {note}")
    else:
        print("Dataset does not meet quality criteria for the specified trait")


Saving normalized gene data...


Normalized gene data saved to ../../output/preprocess/Large_B-cell_Lymphoma/gene_data/GSE248835.csv

Extracting clinical data...
Clinical data saved to ../../output/preprocess/Large_B-cell_Lymphoma/clinical_data/GSE248835.csv
Clinical data shape: (3, 256)
Trait information available: False

Linking clinical and genetic data...
Cannot link data: clinical data is not available

Skipping missing value handling and bias evaluation as linked data is not available

Performing final validation...
Abnormality detected in the cohort: GSE248835. Preprocessing failed.

Dataset usability for Large_B-cell_Lymphoma association studies: False
Reason: Dataset does not contain required trait information
